In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweets-and-user-engagement/Twitterdatainsheets.csv


In [2]:
df=pd.read_csv("/kaggle/input/tweets-and-user-engagement/Twitterdatainsheets.csv")

/tmp/ipykernel_19/2917084621.py:1: DtypeWarning: Columns (3,4,5,6,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("/kaggle/input/tweets-and-user-engagement/Twitterdatainsheets.csv")


## Data pre-processing

In [3]:
df

,index,TweetID,Weekday,Hour,Day,Lang,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,text,LocationID,UserID
0,0,tw-682712873332805633,Thursday,17,31,en,False,44.0,0.0,0.0,35.0,0.0,We are hiring: Senior Software Engineer - Prot...,3751.0,tw-40932430
1,1,tw-682713045357998080,Thursday,17,31,en,True,1810.0,5.0,0.0,53.0,2.0,RT @CodeMineStatus: This is true Amazon Web Se...,3989.0,tw-3179389829
2,2,tw-682713219375476736,Thursday,17,31,en,False,282.0,0.0,0.0,47.0,0.0,Devops Engineer Aws Ansible Cassandra Mysql Ub...,3741.0,tw-4624808414
3,3,tw-682713436967579648,Thursday,17,31,en,False,2087.0,4.0,0.0,53.0,0.0,Happy New Year to all those AWS instances of o...,3753.0,tw-356447127
4,4,tw-682714048199311366,Thursday,17,31,en,False,953.0,0.0,0.0,47.0,0.0,Amazon is hiring! #Sr. #International Tax Mana...,3751.0,tw-3172686669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206290,206290,tw-443876086,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206291,206291,tw-565013710,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206292,206292,tw-322341885,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206293,206293,tw-3304550634,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.columns

Index(['index', 'TweetID', ' Weekday', ' Hour', ' Day', ' Lang', ' IsReshare',
       ' Reach', ' RetweetCount', ' Likes', ' Klout', ' Sentiment', ' text',
       ' LocationID', ' UserID'],
      dtype='object')

In [5]:
df.columns = df.columns.str.replace(' ', '') #the column names have extra spaces on the left

In [6]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x) #some have leading/trailing spaces

In [7]:
df["Weekday"].unique()

array(['Thursday', 'Friday', 'Saturday', 'Sunday', 'Monday', 'Tuesday',
       'Wednesday', nan, 'Country', 'Albania', 'Algeria', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahamas', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Benin',
       'Bhutan', 'Bolivia', 'Bolivia (Plurinational State of)',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Congo', 'Costa Rica', 'Croatia', "Cte d'Ivoire", 'Cuba', 'Cyprus',
       'Czech Republic', 'Democratic Republic of the Congo', 'Denmark',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia',
       'Finland', 'France', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Greenland', 'Guatemala', 'Guernsey', 'Guinea', 'Guyana', 'Haiti',
       'Honduras'

#### As we can see, there is discrepancy with the Weekday column having gender and country values as well. Let us detect this error:

In [8]:
df = df.drop('index', axis=1)

##### Let's first separate the Gender values in a separate dataset.

In [9]:
df[df['Weekday']=="Gender"]

,TweetID,Weekday,Hour,Day,Lang,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,text,LocationID,UserID
106293,UserID,Gender,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Splitting the dataset into two from the row index value:

In [10]:
gender_df=df.iloc[106293:, :]
df=df.iloc[:106293, :]

In [11]:
gender_df

,TweetID,Weekday,Hour,Day,Lang,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,text,LocationID,UserID
106293,UserID,Gender,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106294,tw-1267804344,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106295,tw-27229880,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106296,tw-199664730,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106297,tw-99958381,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206290,tw-443876086,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206291,tw-565013710,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206292,tw-322341885,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206293,tw-3304550634,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Understandably, the only values in the gender dataset is the User IDs and the genders. Let's drop the unnecessary columns. 

In [12]:
gender_df.columns=gender_df.iloc[0]
gender_df=gender_df[1:]
gender_df = gender_df.dropna(axis=1, how='all')
gender_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100001 entries, 106294 to 206294
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   UserID  100000 non-null  object
 1   Gender  100000 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


In [13]:
gender_df=gender_df.reset_index(drop=True)

In [14]:
gender_df=gender_df.dropna(axis=0, how="all")
gender_df

106293,UserID,Gender
0,tw-1267804344,Unknown
1,tw-27229880,Male
2,tw-199664730,Male
3,tw-99958381,Unknown
4,tw-126745533,Male
...,...,...
99995,tw-293303571,Female
99996,tw-443876086,Female
99997,tw-565013710,Unknown
99998,tw-322341885,Female


#### Now, let's separate the Country values.

In [15]:
df[df['Weekday']=="Country"]

,TweetID,Weekday,Hour,Day,Lang,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,text,LocationID,UserID
100001,LocationID,Country,State,StateCode,City,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
country_df=df.iloc[100001:, :]
df=df.iloc[:100001, :]

#### Similar to how we separated the above dataset, we will separate the countries as well.

In [17]:
country_df.columns=country_df.iloc[0]
country_df=country_df[1:]
country_df = country_df.dropna(axis=1, how='all')
country_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6291 entries, 100002 to 106292
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   LocationID  6289 non-null   object
 1   Country     6290 non-null   object
 2   State       6174 non-null   object
 3   StateCode   6180 non-null   object
 4   City        6132 non-null   object
dtypes: object(5)
memory usage: 245.9+ KB


In [18]:
country_df['LocationID'] = country_df['LocationID'].astype(float) #These columns have mixed datatypes in country_df and df(namely, object and float)

In [19]:
country_df=country_df.dropna(axis=0, how="all")
country_df

100001,LocationID,Country,State,StateCode,City
100002,1.0,Albania,Elbasan,AL,Elbasan
100003,2.0,Albania,Tirane,AL,Tirana
100004,3.0,Algeria,Souk Ahras,DZ,Souk Ahras
100005,4.0,Algeria,Alger,DZ,Algiers
100006,5.0,Algeria,Constantine,DZ,Constantine
...,...,...,...,...,...
106287,6286.0,Zambia,Luapula,ZM,Mansa
106288,6287.0,Zambia,Copperbelt,ZM,Ndola
106289,6288.0,Zimbabwe,Harare Province,ZW,Harare
106290,6289.0,Zimbabwe,Bulawayo,ZW,Bulawayo


In [20]:
df=df.dropna(axis=0, how="all")
df[['Hour', 'Day']] = df[['Hour', 'Day']].astype(int) #These columns have mixed datatypes (namely, object and int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0 to 99999
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   TweetID       100000 non-null  object 
 1   Weekday       100000 non-null  object 
 2   Hour          100000 non-null  int64  
 3   Day           100000 non-null  int64  
 4   Lang          100000 non-null  object 
 5   IsReshare     100000 non-null  object 
 6   Reach         100000 non-null  float64
 7   RetweetCount  100000 non-null  float64
 8   Likes         100000 non-null  float64
 9   Klout         100000 non-null  float64
 10  Sentiment     100000 non-null  float64
 11  text          100000 non-null  object 
 12  LocationID    100000 non-null  float64
 13  UserID        100000 non-null  object 
dtypes: float64(6), int64(2), object(6)
memory usage: 11.4+ MB


### We have split the datasets into the respective types. 
### Now, let us merge them with the foreign keys of UserIDs and LocationIDs to create a final dataset

In [21]:
final_df = pd.merge(gender_df, df, on='UserID', how='right')
final_df = pd.merge(country_df, df, on='LocationID', how='right')
final_df

,LocationID,Country,State,StateCode,City,TweetID,Weekday,Hour,Day,Lang,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,text,UserID
0,3751.0,United States,Washington,US-WA,Seattle,tw-682712873332805633,Thursday,17,31,en,False,44.0,0.0,0.0,35.0,0.0,We are hiring: Senior Software Engineer - Prot...,tw-40932430
1,3989.0,United States,Florida,US-FL,Oldsmar,tw-682713045357998080,Thursday,17,31,en,True,1810.0,5.0,0.0,53.0,2.0,RT @CodeMineStatus: This is true Amazon Web Se...,tw-3179389829
2,3741.0,United States,Texas,US-TX,Austin,tw-682713219375476736,Thursday,17,31,en,False,282.0,0.0,0.0,47.0,0.0,Devops Engineer Aws Ansible Cassandra Mysql Ub...,tw-4624808414
3,3753.0,United States,California,US-CA,San Francisco,tw-682713436967579648,Thursday,17,31,en,False,2087.0,4.0,0.0,53.0,0.0,Happy New Year to all those AWS instances of o...,tw-356447127
4,3751.0,United States,Washington,US-WA,Seattle,tw-682714048199311366,Thursday,17,31,en,False,953.0,0.0,0.0,47.0,0.0,Amazon is hiring! #Sr. #International Tax Mana...,tw-3172686669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1349.0,India,Rajasthan,IN,Jaipur,tw-716349486742700032,Saturday,13,2,hi,True,1143.0,1.0,0.0,49.0,1.0,RT @iiManjula: Dekhana thoda dhirese darlingme...,tw-4667163978
99996,1332.0,India,Maharashtra,IN,Mumbai,tw-716349615340199937,Saturday,13,2,en,False,763.0,0.0,0.0,29.0,0.0,I added a video to a @YouTube playlist https:/...,tw-1143330170
99997,1336.0,India,Andhra Pradesh,IN,Hyderabad,tw-716351557516722176,Saturday,13,2,en,True,122.0,435.0,0.0,17.0,1.0,RT @awscloud: Test your iOS Android &amp; web ...,tw-139346924
99998,3795.0,United States,Illinois,US-IL,Moline,tw-716352501860196352,Saturday,13,2,en,True,287.0,30.0,0.0,42.0,0.0,RT @goserverless: Serverless Offline: Emulate ...,tw-21826302


#### Since, this data required a whole lot of cleaning, I will export the dataframe into a CSV file for you guys to use if you would like to perform EDA or build a model on this dataset. :)

In [22]:
final_df.to_csv("tweets_analysis.csv")

## Let's start by visualising a few of our observations:

In [23]:
import plotly.express as px #we will use plotly as it is the most interactive library of Pythong visualisation

In [24]:
# Scatter plot of Reach vs Likes
scatter_fig = px.scatter(final_df, x='Reach', y='Likes', color='Sentiment', title='Reach vs Likes')
scatter_fig.show()

#### The above plot show how most likes are around the reach of 500K, and the sentiment is mostly neutral. The least are negative, but very few negative posts have a higher reach (suggested by the outliers).

In [25]:
# Bar chart of Weekday distribution
bar = px.histogram(final_df, x='Weekday', title='Weekday Distribution')
bar.show()

#### There is not much to infer, but we can see that there are lesser tweets on weekends. 
Probably because weekends are family-time, and gadgets are not allowed ;)

In [26]:
# Line plot showing the frequency of each language

lang_counts = final_df['Lang'].value_counts().reset_index()
lang_counts.columns = ['Language', 'Count']

line_fig = px.line(lang_counts, x='Language', y='Count', markers=True, title='Tweet Frequency per Language')

line_fig.show()

#### The tweets are mostly in english. The rest are plotted in descending order.

In [27]:
# Scatter plot of Klout by Sentiment
kvs = px.scatter(final_df, x='Sentiment', y='Klout', title='Klout by Sentiment')

kvs.show()

#### The sentiment having highest number of data points in the area of klout > 50 is neutral. However, we can infer that negative sentiment tweets have more klout than positive sentiment tweets.

In [28]:
# Group by 'Hour' and calculate the count of users active

user_per_hour = final_df.groupby('Hour')['UserID'].count().reset_index()
user_per_hour['UserCount']=user_per_hour['UserID']
line_user_fig = px.line(user_per_hour, x='Hour', y='UserCount', title='Users active over Time')
line_user_fig.show()

#### Users are most active around 10 AM in the morning, which is a peak. The number of users decrease earlier in the morning or later in the evening.

In [29]:
# Group by 'Day' and calculate the sum of reach

reach_per_day = final_df.groupby('Day')['Reach'].sum().reset_index()

line_reach_fig = px.line(reach_per_day, x='Day', y='Reach', title='Sum of Reach Over a Month')
line_reach_fig.show()

#### There is no particular trend of people being active on Twitter relative to the day of the month.

In [30]:
# Group by 'Country' and calculate the sum of klout

kpc = final_df.groupby('Country')['Klout'].mean().reset_index()
kpc=kpc.sort_values(by='Klout', ascending=False)
kpc=kpc.iloc[:20,:]
kpc

,Country,Klout
16,Botswana,59.000000
80,Maldives,58.333333
108,Republic of the Congo,54.000000
25,Chad,54.000000
75,Luxembourg,52.933333
81,Mali,52.500000
85,Mongolia,49.428571
20,Burundi,46.000000
51,Guinea,45.590909
45,Georgia,45.142857


In [31]:
line_kpc=px.line(kpc, x='Country', y='Klout',markers=True, title='Mean of Klout Of a Country')
line_kpc.show()

#### Most trendy tweets are from Africa (as we can infer by the frequency of African countries in the top 20 countries with highest mean klout of tweets). 
Also, the US does not come in these countries, surprisingly.

In [32]:
final_df['Mood'] = final_df['Sentiment'].apply(lambda x: 'Positive' if x > 0 else ('Neutral' if x == 0 else 'Negative'))
like_per_mood = final_df.groupby('Mood')['Likes'].sum().reset_index()
hist = px.histogram(like_per_mood, x='Mood', y='Likes', color='Mood', title='Likes Over Mood')

# Display the plot
hist.show()

## Let's build a classification model by classifying our tweets in these three categories, namely:

* ### Negative 
* ###  Neutral 
* ###  Positive

#### Let's process the words to efficiently feed into the classification model.

#### First off, our lemmatizers work on English, so let's remove other languages from the dataset.

In [33]:
final_df = final_df[final_df['Lang']=='en']

#### Starting with text processing...

In [34]:
final_df['text']=final_df['text'].astype(str)
final_df['text']=final_df['text'].apply(lambda x:x.lower())

/tmp/ipykernel_19/2047502006.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_19/2047502006.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Stop words are the words that will not contribute to the sentiment of a tweet. Stemming is removing any derivatives and replacing them with the core of the word.

#### 1. Removing the stop words, with the maximum limit of the text being 5000 characters.

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [36]:
vectors= cv.fit_transform(final_df['text']).toarray()

In [37]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

#### 2. Replacing the words with their stems.

In [39]:
import nltk
from nltk.stem.porter import *
ps=PorterStemmer()

In [40]:
def stem(text):
  y=[]
  for i in text.split():
    y.append(ps.stem(i))
  return " ".join(y) #returns string after stemming

In [41]:
final_df['text']= final_df['text'].apply(stem)

/tmp/ipykernel_19/1471107340.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
final_df

,LocationID,Country,State,StateCode,City,TweetID,Weekday,Hour,Day,Lang,IsReshare,Reach,RetweetCount,Likes,Klout,Sentiment,text,UserID,Mood
0,3751.0,United States,Washington,US-WA,Seattle,tw-682712873332805633,Thursday,17,31,en,False,44.0,0.0,0.0,35.0,0.0,we are hiring: senior softwar engin - proto ht...,tw-40932430,Neutral
1,3989.0,United States,Florida,US-FL,Oldsmar,tw-682713045357998080,Thursday,17,31,en,True,1810.0,5.0,0.0,53.0,2.0,rt @codeminestatus: thi is true amazon web ser...,tw-3179389829,Positive
2,3741.0,United States,Texas,US-TX,Austin,tw-682713219375476736,Thursday,17,31,en,False,282.0,0.0,0.0,47.0,0.0,devop engin aw ansibl cassandra mysql ubuntu r...,tw-4624808414,Neutral
3,3753.0,United States,California,US-CA,San Francisco,tw-682713436967579648,Thursday,17,31,en,False,2087.0,4.0,0.0,53.0,0.0,happi new year to all those aw instanc of ours!,tw-356447127,Neutral
4,3751.0,United States,Washington,US-WA,Seattle,tw-682714048199311366,Thursday,17,31,en,False,953.0,0.0,0.0,47.0,0.0,amazon is hiring! #sr. #intern tax manag - aw ...,tw-3172686669,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,611.0,China,Beijing,CN,Beijing,tw-716349383688720385,Saturday,13,2,en,True,415.0,15.0,0.0,44.0,0.0,rt @awscloud: .@jaspersoft built a #datawareho...,tw-3316009302,Neutral
99996,1332.0,India,Maharashtra,IN,Mumbai,tw-716349615340199937,Saturday,13,2,en,False,763.0,0.0,0.0,29.0,0.0,i ad a video to a @youtub playlist https://www...,tw-1143330170,Neutral
99997,1336.0,India,Andhra Pradesh,IN,Hyderabad,tw-716351557516722176,Saturday,13,2,en,True,122.0,435.0,0.0,17.0,1.0,rt @awscloud: test your io android &amp; web a...,tw-139346924,Positive
99998,3795.0,United States,Illinois,US-IL,Moline,tw-716352501860196352,Saturday,13,2,en,True,287.0,30.0,0.0,42.0,0.0,rt @goserverless: serverless offline: emul aw ...,tw-21826302,Neutral


#### Now that we are done with the text processing, let us label the moods with integer value (1, 2, 3) using the Label Encoder object. We will use this in a pipeline for further classification.

In [43]:
le_model = LabelEncoder()
final_df['Label'] = le_model.fit_transform(final_df['Mood'])

/tmp/ipykernel_19/2482168643.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Training model:

#### Splitting the dataset into testing and training sets:

In [44]:
X_train, X_test, y_train, y_test = train_test_split(final_df['text'], final_df['Label'], 
                                                    test_size=0.2, random_state=42, stratify=final_df['Label'])

In [45]:
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

Shape of X_train:  (73508,)
Shape of X_test:  (18378,)


### Creating a Pipeline classifier:
#### So, why are we using a Pipeline? More than that, why are we taking the Random Forest in the pipeline with the Term Frequency-Inverse Document Frequency (TF-IDF) vectorizer?

##### The TF-IDF vectorizer is often used in tasks of Natural Language Processing (NLP), as it tokenises the words into various vectors that act as features to the TF-IDF matrix. The Random Forest Classifier is used in a pipeline with it since our data is extremely uneven. There is a very high number of neutral sentiment tweets, which is not comparable to the negative tweets. The random forest utilises decision trees, which is more efficient.

In [46]:
clf = Pipeline([
    ('vectorizer_tri_grams', TfidfVectorizer()),
    ('random_forest', (RandomForestClassifier()))   
])

In [47]:
# Model training
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer_tri_grams', TfidfVectorizer()),
                ('random_forest', RandomForestClassifier())])

In [48]:
# Get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)

In [49]:
# Print score
print(accuracy_score(y_test, y_pred))

0.9099466753727282


In [50]:
# Print classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.52      0.68      1023
           1       0.89      0.99      0.94     12274
           2       0.96      0.80      0.87      5081

    accuracy                           0.91     18378
   macro avg       0.94      0.77      0.83     18378
weighted avg       0.92      0.91      0.90     18378



### The tweets will be classified into different moods with an accuracy of 91.02%.

### Testing our model:

In [51]:
test_df=final_df.iloc[:50000, :]

In [52]:
test=df['text'][1000]

Above is the original tweet we will use for processing below:

In [53]:
test_text = test_df['text'][1000]
print(f"{test} ===> {test_df['Label'][1000]}")

RT @datadoghq: The Top 5 AWS EC2 Performance Problems: https://cards.twitter.com/cards/21ablq/15bk2 ===> 1


In [54]:
test = clf.predict([test_text])

In [55]:
classes = ['Negative', 'Neutral', 'Positive']

print(f"True Label: {test_df['Mood'][1000]}")
print(f'Predict Label: {classes[test[0]]}')

True Label: Neutral
Predict Label: Neutral


#### We have successfully predicted the mood of the tweets! 